In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.metrics import classification_report, confusion_matrix
import plotly.io as pio
from scipy import stats
from scipy.stats import chi2_contingency, normaltest, mannwhitneyu

# configuración de visualización 
plt.style.use('seaborn-v0_8-dark')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_palette('husl')


-----CARGA DE DATOS

In [84]:
df_bank_loan = pd.read_excel('Bankloan.xlsx', sheet_name='Hoja2')
print(f'''
      Dataframe de Préstamo Bancario:\n\n
      Estadística Básica:\n {df_bank_loan.describe()}\n
      Nombre de las columnas:\n {df_bank_loan.columns}\n 
      Información general del dataframe:\n {df_bank_loan.info}\n\n
      Primera filas:\n {df_bank_loan.head(10)}\n 
      Últimas filas:\n {df_bank_loan.tail(10)}\n
      Tamaño del Dataframe: {df_bank_loan.shape}
      
      ''')


      Dataframe de Préstamo Bancario:


      Estadística Básica:
               age          ed      employ     address        income  \
count  681.000000  680.000000  700.000000  700.000000  6.630000e+02   
mean    34.898678    1.717647    8.388571    8.268571  4.574359e+07   
std      8.861849    0.925652    6.658039    6.821609  3.744108e+07   
min     20.000000    1.000000    0.000000    0.000000  1.400000e+07   
25%     28.000000    1.000000    3.000000    3.000000  2.400000e+07   
50%     34.000000    1.000000    7.000000    7.000000  3.400000e+07   
75%     40.000000    2.000000   12.000000   12.000000  5.450000e+07   
max    136.000000    5.000000   31.000000   34.000000  4.460000e+08   

            debtinc      creddebt       othdebt  
count  7.000000e+02  7.000000e+02  7.000000e+02  
mean   1.026057e+07  1.553553e+06  3.058209e+06  
std    6.827234e+06  2.117197e+06  3.287555e+06  
min    4.000000e+05  1.169600e+04  4.558400e+04  
25%    5.000000e+06  3.690592e+05  1.04417

In [85]:
# tratamiento de valores nulos
df_nulls = df_bank_loan.isnull().sum()
print(f'Total de valores nulos:\n {df_nulls[df_nulls > 0]}')

Total de valores nulos:
 age       19
ed        20
income    37
dtype: int64


----- TRATAMIENTO DE VALORES NULOS

In [86]:
# Rellenar valores nulos
df_bank_loan['age']=df_bank_loan['age'].where(df_bank_loan['age'] < 60)
df_bank_loan['age'].fillna(df_bank_loan['age'].median())
df_bank_loan['ed'].fillna(df_bank_loan['ed'].mode()[0])
df_bank_loan['income'].fillna(df_bank_loan['income'].median())

# conversión a columnas numéricas
df_bank_loan['default']=pd.to_numeric(df_bank_loan['default'], errors='coerce')
df_bank_loan['default'].fillna(0)


0      1.0
1      0.0
2      0.0
3      0.0
4      1.0
      ... 
695    1.0
696    0.0
697    0.0
698    0.0
699    0.0
Name: default, Length: 700, dtype: float64

----- ANÁLSIS UNIVARIANTE

In [87]:
def univariante_analysis(df):
    ''' Función de análisis univariante'''
    print('=== Análisis Univariante===')
    
    # Estadística Descriptiva
    print(f'\nEstadística descriptivas:\n {df_bank_loan.describe()}')
    
    # columna de edad transformada
    ed_map={
        1: "Postgrado / Máster / Doctorado",
        2: "Universidad / Grado",
        3: "Secundaria / Bachillerato",
        4: "Primaria",
        5: "Sin estudios"
    }
    df_bank_loan['ed_text'] = df_bank_loan['ed'].map(ed_map)
    # Distribución de variables numéricas
    numeric_cols = ['age', 'ed_text', 'employ', 'address', 'income', 'debtinc', 'creddebt', 'othdebt']

    fig = px.histogram(df_bank_loan, x='age', nbins=30, title='Distribucion de Edad')
    fig.show()    

    # variables categóricas
    fig = px.pie(df_bank_loan, names='ed_text', title='Distribució')
    
univariante_analysis(df_bank_loan)

=== Análisis Univariante===

Estadística descriptivas:
               age          ed      employ     address        income  \
count  680.000000  680.000000  700.000000  700.000000  6.630000e+02   
mean    34.750000    1.717647    8.388571    8.268571  4.574359e+07   
std      7.973215    0.925652    6.658039    6.821609  3.744108e+07   
min     20.000000    1.000000    0.000000    0.000000  1.400000e+07   
25%     28.000000    1.000000    3.000000    3.000000  2.400000e+07   
50%     34.000000    1.000000    7.000000    7.000000  3.400000e+07   
75%     40.000000    2.000000   12.000000   12.000000  5.450000e+07   
max     56.000000    5.000000   31.000000   34.000000  4.460000e+08   

            debtinc      creddebt       othdebt     default  
count  7.000000e+02  7.000000e+02  7.000000e+02  698.000000  
mean   1.026057e+07  1.553553e+06  3.058209e+06    0.262178  
std    6.827234e+06  2.117197e+06  3.287555e+06    0.440134  
min    4.000000e+05  1.169600e+04  4.558400e+04    0.000

In [88]:
def univariante_analysis(df):
    ''' Función de análisis univariante'''
    print('=== Análisis Univariante===')
    
    # Estadística Descriptiva
    print(f'\nEstadística descriptivas:\n {df_bank_loan.describe()}')

    # Distribución de variables numéricas
    numeric_cols = ['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt', 'othdebt']

    fig = px.histogram(df_bank_loan, x='age', nbins=30, title='Distribucion de Edad')
    fig.show()    

    # variables categóricas
    fig = px.pie(df_bank_loan, names='ed_text', title='Distribución por Nivel Educativo')
    fig.show()
    
univariante_analysis(df_bank_loan)

=== Análisis Univariante===

Estadística descriptivas:
               age          ed      employ     address        income  \
count  680.000000  680.000000  700.000000  700.000000  6.630000e+02   
mean    34.750000    1.717647    8.388571    8.268571  4.574359e+07   
std      7.973215    0.925652    6.658039    6.821609  3.744108e+07   
min     20.000000    1.000000    0.000000    0.000000  1.400000e+07   
25%     28.000000    1.000000    3.000000    3.000000  2.400000e+07   
50%     34.000000    1.000000    7.000000    7.000000  3.400000e+07   
75%     40.000000    2.000000   12.000000   12.000000  5.450000e+07   
max     56.000000    5.000000   31.000000   34.000000  4.460000e+08   

            debtinc      creddebt       othdebt     default  
count  7.000000e+02  7.000000e+02  7.000000e+02  698.000000  
mean   1.026057e+07  1.553553e+06  3.058209e+06    0.262178  
std    6.827234e+06  2.117197e+06  3.287555e+06    0.440134  
min    4.000000e+05  1.169600e+04  4.558400e+04    0.000

----- ANÁLISIS BIVARIANTE

In [89]:
def bivariante_analysis(df):
    print('=== Análisis Biriante===')
    
    default_cat= {
        0: 'Aprobado',
        1: 'No Aprobado'
    }
    # Crear una copia para no modificar el original
    df_plot = df_bank_loan.copy()
    df_plot['default_mod'] = df_plot['default'].map(default_cat)
    
    # Correlación entre variables 
    numeric_cols = df_bank_loan.select_dtypes(include=[np.number]).columns.tolist()
    corr_matrix = df_bank_loan[numeric_cols].corr()
    corr_matrix = corr_matrix.rename(columns={
    'age':'Edad',
    'ed':'Educación',
    'employ':'Empleo',
    'adress':'Dirección',
    'income':'Ingreso',
    'debtinc':'Deuda',
    'creddebt':'Deuda Crédito',
    'othdebt':'Otra Deuda',
    'default':'Incumplimiento'
    }, index={
        'age':'Edad',
        'ed':'Educación',
        'employ':'Empleo',
        'adress':'Dirección',
        'income':'Ingreso',
        'debtinc':'Deuda',
        'creddebt':'Deuda Crédito',
        'othdebt':'Otra Deuda',
        'default':'Incumplimiento'
    })

    
    fig = px.imshow(
        corr_matrix,
        text_auto=True,
        aspect='auto',
        title='Mapa de Calor de Correlaciones',
        color_continuous_scale=['red','white','green'],
        zmin=-1,
        zmax=1,
    )   
    fig.update_traces(
        text=np.round(corr_matrix.values,2),
        texttemplate='%{text}',
        textfont=dict(size=10, color='black'))  
    fig.show()
    
    # Relacion entre edad e ingresos
    fig = px.scatter(
        df_plot,x='age', y='income', color='default_mod',
        title='Edad VS Ingresos por Estado de Default',
        labels={'default_mod': 'Estado del Préstamo', 'age': 'Edad', 'income': 'Ingresos'},
        color_continuous_scale=['green','red'],
       
    )
    fig.show()
        
    #boxplot de ingresos por defeault
    fig = px.box(df_plot, x='default_mod', y='income',

                 title='Distribucón de Ingresos po Estado de Default'
    )
    fig.show()

    # Violin de deuda por educación
    fig = px.violin(
        df_bank_loan, x='ed_text', y='creddebt',
        title='Distribución de Deuda Crediticia por Nivel Educativo'
    )
    fig.show()
    
bivariante_analysis(df_bank_loan)


=== Análisis Biriante===


----- ANÁLISIS MULTIVARIANTE

In [90]:
def multivariante_analysis(df):
    print('=== Análisis Multivariante===')

    # Crear una copia para no modificar el original
    df_plot = df_bank_loan.copy()

    # Crear una columna de categorías para el gráfico de dispersión
    df_plot['default_cat'] = df_plot['default'].map({0: 'Aprobado', 1: 'No Aprobado'})

    # Gráfico de dispersión
    fig = px.scatter_matrix(
        df_plot,
        dimensions=['age', 'employ', 'income', 'creddebt', 'othdebt'],
        color='default_cat',
        title='Gráfico de Dispersión Multivariante',
        labels={'default_cat': 'Estado del Préstamo', 'age': 'Edad', 'employ': 'Empleo', 'income': 'Ingresos', 'creddebt': 'Deuda Crédito', 'othdebt': 'Otra Deuda'},
        color_discrete_sequence=['red','green'],
        width=900,
        height=700
    )
    fig.show()

    # Gráfico de dispersión 3D
    fig = px.scatter_3d(
        df_plot,
        x='age',
        y='income',
        z='creddebt',
        color='default_cat',
        title='Gráfico de Dispersión 3D',
        labels={'default_cat': 'Estado del Préstamo', 'age': 'Edad', 'income': 'Ingresos', 'creddebt': 'Deuda Crédito'},
        color_discrete_sequence=['red','green'],
        height=600,
        
    )
    
    fig.update_layout(
        scene = dict(
            xaxis = dict(showgrid=True, gridwidth=1, title_font=dict(size=14)),
            yaxis = dict(showgrid=True, gridwidth=1, title_font=dict(size=14)),
            zaxis = dict(showgrid=True, gridwidth=1, title_font=dict(size=14)),
            camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=2.5, y=1.5, z=1.5)
            ),
        ),
        margin=dict(l=20, r=0, t=20, b=0),
        scene_aspectmode='cube'
    )
    fig.show()

multivariante_analysis(df_bank_loan)

=== Análisis Multivariante===


----- A/B TESTING

In [91]:
def ab_testing(df):
    print('=== A/B Testing===')
    # Crear una copia para no modificar el original
    df_plot = df_bank_loan.copy()
    
    #  Crear una columna de categorías para el gráfico de dispersión
    df_plot['default_cat'] = df_plot['default'].map({0: 'Aprobado', 1: 'No Aprobado'})
    
    # dividir en grupos
    default_0 = df_plot[df_plot['default'] == 0]
    default_1 = df_plot[df_plot['default'] == 1]
    
    # test t para ingresos
    t_statistic, p_value = stats.ttest_ind(default_0['income'], default_1['income'])
    print(f'Test t para Ingresos: t-statistic={t_statistic}, p-value={p_value:.3f}')
   
   # test t para edad
    t_statistic, p_value = stats.ttest_ind(default_0['age'], default_1['age'])
    print(f'Test t para Edad: t-statistic={t_statistic}, p-value={p_value:.3f}')

    # test t para empleo
    t_statistic, p_value = stats.ttest_ind(default_0['employ'], default_1['employ'])
    print(f'Test t para Empleo: t-statistic={t_statistic}, p-value={p_value:.3f}')

    # test t para deuda
    t_statistic, p_value = stats.ttest_ind(default_0['creddebt'], default_1['creddebt'])
    print(f'Test t para Deuda: t-statistic={t_statistic}, p-value={p_value:.3f}')

    # test t para otra deuda
    t_statistic, p_value = stats.ttest_ind(default_0['othdebt'], default_1['othdebt'])
    print(f'Test t para Otra Deuda: t-statistic={t_statistic}, p-value={p_value:.3f}')
    
    # chi-squared para educación
    contingency_table = pd.crosstab(df_plot['ed'], df_plot['default'])  
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    print(f'Chi-squared para Educación: chi2={chi2:.3f}, p-value={p_value:.3f}')
    print(f'Grados de libertad: {dof}')
    print(f'Valores esperados:\n{expected}')
    
    # Distribución por niver educativo
    fig= px.histogram(df_plot, x='ed_text', color='default_cat', barmode='group',
                       title='Distribución por Nivel Educativo',
                       labels={'default_cat': 'Estado del Préstamo', 'ed_text': 'Nivel Educativo'},
                       color_discrete_sequence=['red','green'],
                       width=900,
                       height=700
    )
    fig.update_layout(
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        )
    )
    fig.show()
    
    # distribución por empleo
    fig = px.histogram(df_plot, x='employ', color='default_cat', barmode='group',
                       title='Distribución por Empleo',
                       labels={'default_cat': 'Estado del Préstamo', 'employ': 'Empleo'},
                       color_discrete_sequence=['red','green'],
                       width=900,
                       height=700
    )
    fig.update_layout(
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        )
    )
    fig.show()
ab_testing(df_bank_loan) 

=== A/B Testing===
Test t para Ingresos: t-statistic=nan, p-value=nan
Test t para Edad: t-statistic=nan, p-value=nan
Test t para Empleo: t-statistic=7.758771768486403, p-value=0.000
Test t para Deuda: t-statistic=-6.649253570848013, p-value=0.000
Test t para Otra Deuda: t-statistic=-3.8775829228348297, p-value=0.000
Chi-squared para Educación: chi2=14.090, p-value=0.007
Grados de libertad: 4
Valores esperados:
[[266.96165192  95.03834808]
 [140.85545723  50.14454277]
 [ 61.94690265  22.05309735]
 [ 26.54867257   9.45132743]
 [  3.68731563   1.31268437]]


----- ANÁLISIS AVANZADO CON VISUALIZACIONES

In [137]:
def advance_visualizations(df):
    print('=== Análisis Avanzado===')
    
    print("Valores nulos en income:", df_bank_loan['income'].isnull().sum())
    print("Valores nulos en ed_text:", df_bank_loan['ed_text'].isnull().sum())
    
    
    # Treemap de ingresos por educacion y default
    df_plot = df_bank_loan.copy()
    df_plot['default_cat'] = df_plot['default'].map({0: 'Aprobado', 1: 'No Aprobado'})
    
    df_plot = df_plot.dropna(subset=['income', 'ed_text', 'default_cat'])
    
    fig=px.treemap(df_plot, path=['ed_text', 'default_cat'], values='income',
                   title='Ingresos por Nivel Educativo y Estado de Default',
                   color='default_cat',
                   color_discrete_sequence=["#4E2D30","#2E5C2E"],
                   width=800,
                   height=600
    )
    
    fig.update_traces(
        textposition="middle center",
        textfont_size=12,
        textinfo="label+value",
        textfont_color='white',
        marker=dict(
            line=dict(width=2, color='DarkSlateGrey')
        )
    )
    fig.update_layout(
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1.02,
            xanchor='right',
            x=1
        )
    )
    fig.show()

    
advance_visualizations(df_bank_loan)

=== Análisis Avanzado===
Valores nulos en income: 37
Valores nulos en ed_text: 20
